# ***Kaggle:*** Chess Evaluations

| Author's Name         | NIU     |
| --------------------- | ------- |
| Albert Capdevila      | 1587933 |

<small>*This Jupyter Notebook is submitted as an assignment for the Machine Learning subject in the Computer Engineering degree at the Universitat Autònoma de Barcelona (UAB).*<small>

## 1. Problem definition

In computer chess, an evaluation function estimates the strength of a position by assigning a numerical score that indicates which side (White or Black) has the advantage and by how much.

This score is based on the values of chess pieces, which are traditionally:
| Piece  | Pawn | Knight | Bishop | Rook | Queen |
|--------|------|--------|--------|------|-------|
| Symbol | ![Pawn](https://upload.wikimedia.org/wikipedia/commons/thumb/4/45/Chess_plt45.svg/45px-Chess_plt45.svg.png) | ![Knight](https://upload.wikimedia.org/wikipedia/commons/thumb/7/70/Chess_nlt45.svg/45px-Chess_nlt45.svg.png) | ![Bishop](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/Chess_blt45.svg/45px-Chess_blt45.svg.png) | ![Rook](https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/Chess_rlt45.svg/45px-Chess_rlt45.svg.png) | ![Queen](https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Chess_qlt45.svg/45px-Chess_qlt45.svg.png) |
| Value  | 1    | 3      | 3      | 5    | 9     |

<small>*The value of the king is undefined as it cannot be captured, let alone traded, during the course of the game. Chess engines usually assign the king an arbitrary large value such as 200 points [¹](#references).*</small>

These values show that, for example, capturing a bishop is equivalent to capturing three pawns. Therefore, all of them can be expressed in *'pawn units'*.

It’s also important to note that evaluations are opposite, with positive values indicating an advantage for White and negative values for Black. For example, a negative evaluation of $-3$ pawns may indicate that Black has an advantage equivalent to three pawns, a bishop, or a knight.

Usually, these evaluations are calculated by complex algorithms, such as *Stockfish*, which are based on **minimax** principles, calculating, pruning, and predicting the best moves across multiple levels of depth. These algorithms can be slow and computationally intensive, as they analyze many possible future moves in a position. This notebook will explore the use of regression models to achieve similar results more quickly.

***In this notebook, we aim to estimate the evaluation score of a chess position through feature extraction and simple regression techniques.***



## 2. Data collection

For this machine learning problem, we will use the **Chess Evaluations** dataset, available [here](https://www.kaggle.com/datasets/ronakbadhe/chess-evaluations/data) on Kaggle. The dataset contains two columns: 

| Feature             | Description                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Position (FEN)**  | This is a string representing the chess position in Forsyth-Edwards Notation (FEN). |
| **Evaluation**      | This is the evaluation of the position in ***centipawns***. `#` means that there is forced checkmate. |

The evaluations are in centi-pawns and are generated from Stockfish 11 at depth 22.

Let’s download and import the dataset:

In [91]:

from kaggle.api.kaggle_api_extended import KaggleApi
from zipfile import ZipFile
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import os 

api = KaggleApi()
api.authenticate()

dataser_owner = 'ronakbadhe'
dataset_name= 'chess-evaluations'
dataset = f'{dataser_owner}/{dataset_name}'
target = 'Evaluation'

if not os.path.isdir('data'):
  print("The 'data' directory was not found")
  print("Downloading the dataset from Kaggle...")
  os.mkdir('data')
  api.dataset_download_files(dataset = dataset,path = 'data')
  zf = ZipFile(f'data/{dataset_name}.zip')
  zf.extractall(f'data')
  zf.close()
  os.remove(f'data/{dataset_name}.zip')
  print("Dataset downloaded successfully ✓")
else:
  print("The 'data' directory was found successfully ✓")

The 'data' directory was found successfully ✓


In [92]:
csvFile = "chessData.csv"
print(f"Reading the file '{csvFile}'...")

df = pd.read_csv(f'data/{csvFile}')

print(f"File '{csvFile}' read successfully ✓")

Reading the file 'chessData.csv'...
File 'chessData.csv' read successfully ✓


In [93]:
# Reducing the dataset size to make it faster to work with
from sklearn.model_selection import train_test_split

_, df = train_test_split(df, test_size=0.001, random_state=42)

In [94]:
df.head()

,FEN,Evaluation
11784899,r3k2r/1b2bppp/p1n1pn2/1p2N1B1/8/2NB1P2/PPP3PP/...,-35
8860705,r3k2r/1pp2p2/1bnp1q2/p2Bp1p1/PP2P1bP/1QPP1N2/3...,-201
7376649,6k1/2p3pp/8/8/P3R3/3r1PP1/5K1P/8 b - - 0 30,+121
2224083,1r6/2rnppkp/1q1p2n1/2pP2p1/2b5/1P4PP/P1QNPPB1/...,+280
4108821,5k2/3R4/1p1p4/3n2pp/8/8/PP6/6K1 b - - 1 35,+328


## 3. Data preprocessing

### 3.1 Missing Data

In [95]:
df.isna().sum()

FEN           0
Evaluation    0
dtype: int64

Since there are no missing values, there is no need to handle NaN values.

### 3.2 Data Conversion

In [96]:
df.dtypes

FEN           object
Evaluation    object
dtype: object

The evaluation feature is a string that starts with either a "+" or "-" symbol. To use it in the model, it will need to be converted into a numeric value.

It may also include the character `#`, but since this is not relevant to the current problem, we will ignore it.

In [97]:
tqdm.pandas(desc="Converting 'Evaluation' to numeric values")
df['Evaluation'] = df['Evaluation'].progress_apply(lambda x: int(x) if '#' not in x else int(x.replace('#', '')))

Converting 'Evaluation' to numeric values: 100%|██████████| 12959/12959 [00:00<00:00, 518338.25it/s]


In [98]:
df.dtypes

FEN           object
Evaluation     int64
dtype: object

### 3.3 Data Normalization

Apart from the target feature (Evaluation), the dataset contains only one feature: FEN, which cannot be normalized. Therefore, normalization is not applicable in this case.

### 3.4 Feature extraction

In this section, we will focus on feature extraction, which is the most important step in building our model to predict the evaluation of a chess position. At the moment, we only have the FEN notation of the chess position, and our goal is to extract useful features from it.

#### 3.4.1 Information in the FEN Notation

Forsyth–Edwards Notation (FEN) is a standard notation for describing a particular board position of a chess game.

A typical FEN string looks like this: `rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1`

The FEN string above represents the following board position:

<img src="img/example_board.svg" alt="Example Board" width="300"/>

A FEN string has six fields, each separated by a space:

1. **Piece placement data:**

    It is the arrangement of pieces on the board, row by row, from top to bottom. Each piece is represented by a letter (uppercase for White and lowercase for Black). Empty squares are indicated by numbers.

2. **Active color:**

    Indicates which side is to move (`w` for White, `b` for Black).

3. **Castling availability:**

    If no castling is possible, this field is `-`. Otherwise, it shows `K`, `Q`, `k`, or `q` for available castling options.

4. **En passant target square:**

    Indicates the square where an en passant capture is possible, or `-` if not.

5. **Halfmove clock:**

    Counts the number of halfmoves since the last pawn advance or capture, used for the 50-move rule. 

6. **Fullmove number:**

    Tracks the total number of moves in the game, starting at 1 and incrementing after each Black move.

Only using this information, we can start extracting some simple new features:

In [99]:
def count_white_pieces(fen):
    return len(re.findall(r'[KQRBNP]', fen))

def count_black_pieces(fen):
    return len(re.findall(r'[kqrbnp]', fen))

tqdm.pandas(desc="Adding column 'White_pieces_count'")
df['White_pieces_count'] = df['FEN'].progress_apply(count_white_pieces)
tqdm.pandas(desc="Adding column 'Black_pieces_count'")
df['Black_pieces_count'] = df['FEN'].progress_apply(count_black_pieces)

Adding column 'Black_pieces_count': 100%|██████████| 12959/12959 [00:00<00:00, 134915.60it/s]


In [100]:
piece_values = {
    'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9,
    'p': -1, 'n': -3, 'b': -3, 'r': -5, 'q': -9
}

# The king's value is ignored for now.

def get_white_material(fen):
    return sum(piece_values[piece] for piece in re.findall(r'[QRBNP]', fen))

def get_black_material(fen):
    return -sum(piece_values[piece] for piece in re.findall(r'[qrbnp]', fen))

tqdm.pandas(desc="Adding column 'White_material'")
df['White_material'] = df['FEN'].progress_apply(get_white_material)

tqdm.pandas(desc="Adding column 'Black_material'")
df['Black_material'] = df['FEN'].progress_apply(get_black_material)

df["Material_difference"] = df['White_material'] - df['Black_material']
print("Added column 'Material_difference' ✓")

Adding column 'Black_material': 100%|██████████| 12959/12959 [00:00<00:00, 118860.83it/s]

Added column 'Material_difference' ✓


In [101]:
def get_turn(fen):
    return 1 if fen.split(' ')[1] == 'w' else -1

tqdm.pandas(desc="Adding column 'Turn'")
df['Turn'] = df['FEN'].progress_apply(get_turn)

Adding column 'Turn': 100%|██████████| 12959/12959 [00:00<00:00, 417853.65it/s]


In [102]:
def get_castling(fen, char):
    return int(char in fen.split(' ')[2])

tqdm.pandas(desc="Adding column 'White_king_castling'")
df['White_king_castling'] = df['FEN'].progress_apply(lambda fen: get_castling(fen, 'K'))

tqdm.pandas(desc="Adding column 'White_queen_castling'")
df['White_queen_castling'] = df['FEN'].progress_apply(lambda fen: get_castling(fen, 'Q'))

tqdm.pandas(desc="Adding column 'Black_king_castling'")
df['Black_king_castling'] = df['FEN'].progress_apply(lambda fen: get_castling(fen, 'k'))

tqdm.pandas(desc="Adding column 'Black_queen_castling'")
df['Black_queen_castling'] = df['FEN'].progress_apply(lambda fen: get_castling(fen, 'q'))

df['Castling_difference'] = df['White_king_castling'] + df['White_queen_castling'] - df['Black_king_castling'] - df['Black_queen_castling']
print("Added column 'Castling_difference' ✓")


Adding column 'Black_queen_castling': 100%|██████████| 12959/12959 [00:00<00:00, 462854.98it/s]

Added column 'Castling_difference' ✓


In [103]:
def get_en_passant(fen):
    # This feature is 1 if en passant is available for White, -1 for Black, and 0 if none
    return int(fen.split(' ')[3] != '-') * 1 if fen.split(' ')[1] == 'w' else -1

tqdm.pandas(desc="Adding column 'En_passant'")
df['En_passant'] = df['FEN'].progress_apply(get_en_passant)

Adding column 'En_passant': 100%|██████████| 12959/12959 [00:00<00:00, 350388.30it/s]


In [104]:
def get_rule50(fen):
    return int(fen.split(' ')[4]) * 1 if fen.split(' ')[1] == 'w' else -1

tqdm.pandas(desc="Adding column 'Rule_50'")
df['Rule_50'] = df['FEN'].progress_apply(get_rule50)

Adding column 'Rule_50': 100%|██████████| 12959/12959 [00:00<00:00, 350395.08it/s]


In [105]:
def get_n_moves(fen):
    return int(fen.split(' ')[5]) + int(fen.split(' ')[1] == 'b')

tqdm.pandas(desc="Adding column 'N_moves'")
df['N_moves'] = df['FEN'].progress_apply(get_n_moves)

Adding column 'N_moves': 100%|██████████| 12959/12959 [00:00<00:00, 294526.52it/s]


Let's see the correlations between the new features and the target variable:

In [106]:
correlations = df.corr(numeric_only=True)['Evaluation']
sorted_correlations = correlations.reindex(correlations.abs().sort_values(ascending=False).index)
print("Correlations with 'Evaluation' (sorted by absolute value):")
print(sorted_correlations)

Correlations with 'Evaluation' (sorted by absolute value):
Evaluation              1.000000
Material_difference     0.195666
White_pieces_count      0.052898
White_material          0.036804
Black_pieces_count     -0.031224
En_passant              0.028921
Turn                    0.028843
Rule_50                 0.021402
Black_material         -0.017215
N_moves                -0.007546
Black_king_castling     0.001712
Castling_difference    -0.001312
White_king_castling    -0.000689
White_queen_castling   -0.000240
Black_queen_castling   -0.000061
Name: Evaluation, dtype: float64


There are no significant correlations.

Currently, we only extract information from the FEN string, not from the position itself (for instance, two positions with the same pieces, turn, en passant, and number of moves would appear identical to the model).

We need to extract information directly from the game. To achieve this, we'll use the chess library to obtain a board representation and apply some of its functions to build new features.

In [107]:
%pip install chess
import chess

Note: you may need to restart the kernel to use updated packages.


In [108]:
def get_board(fen):
    return chess.Board(fen)

tqdm.pandas(desc="Adding column 'Board'")
df['Board'] = df['FEN'].progress_apply(get_board)

Adding column 'Board': 100%|██████████| 12959/12959 [00:01<00:00, 9106.55it/s]


Here are a few features I think could have a strong correlation with the evaluation:

* **Attack count:**
    It is the number of squares attacked by each individual piece of a player. If a square is attacked by more than one piece, it is counted multiple times.

In [109]:
def get_attacks_count(board, color):
    return sum(len(board.attackers(color, square)) for square in chess.SQUARES)

tqdm.pandas(desc="Adding column 'White_attacks'")
df['White_attacks'] = df['Board'].progress_apply(lambda board: get_attacks_count(board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_attacks'")
df['Black_attacks'] = df['Board'].progress_apply(lambda board: get_attacks_count(board, chess.BLACK))

df["Attacks_difference"] = df['White_attacks'] - df['Black_attacks']
print("Added column 'Attacks_difference' ✓")

Adding column 'Black_attacks': 100%|██████████| 12959/12959 [00:02<00:00, 5250.80it/s]

Added column 'Attacks_difference' ✓


* **Central attack count**: It is the number of attacks on the central squares (d4, e4, d5, e5) by each individual piece of a player. If a square is attacked by more than one piece, it is counted multiple times. The central squares are often key in chess, as controlling them typically gives strategic advantages.

In [110]:
def get_central_attacks_count(board, color):
    central_squares = [chess.D4, chess.E4, chess.D5, chess.E5]
    return sum(len(board.attackers(color, square)) for square in central_squares)

tqdm.pandas(desc="Adding column 'White_central_attacks'")
df['White_central_attacks'] = df['Board'].progress_apply(lambda board: get_central_attacks_count(board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_central_attacks'")
df['Black_central_attacks'] = df['Board'].progress_apply(lambda board: get_central_attacks_count(board, chess.BLACK))
df["Central_attacks_difference"] = df['White_central_attacks'] - df['Black_central_attacks']
print("Added column 'Central_attacks_difference' ✓")

Adding column 'Black_central_attacks': 100%|██████████| 12959/12959 [00:00<00:00, 63838.62it/s]

Added column 'Central_attacks_difference' ✓


* **Legal moves:**
    It is the total number of legal moves available for a player in the current position.

In [111]:
def get_legal_moves(board, color):
    orig_turn = board.turn 
    board.turn = color
    legal_moves = len(list(board.legal_moves))
    board.turn = orig_turn
    return legal_moves

tqdm.pandas(desc="Adding column 'White_legal_moves'")
df['White_legal_moves'] = df['Board'].progress_apply(lambda board: get_legal_moves(board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_legal_moves'")
df['Black_legal_moves'] = df['Board'].progress_apply(lambda board: get_legal_moves(board, chess.BLACK))

df["Legal_moves_difference"] = df['White_legal_moves'] - df['Black_legal_moves']
print("Added column 'Legal_moves_difference' ✓")

Adding column 'Black_legal_moves': 100%|██████████| 12959/12959 [00:01<00:00, 10622.31it/s]

Added column 'Legal_moves_difference' ✓


* **Attacked pieces**:
    It is the total number of opponent attacks on squares occupied by a player's pieces.

In [112]:
def get_attacked_pieces(board, color):
    attacked_pieces = 0
    opponent_color = chess.WHITE if color == chess.BLACK else chess.BLACK 
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece and piece.color == color:
            attacked_pieces += len(board.attackers(opponent_color, square))

    return attacked_pieces

tqdm.pandas(desc="Adding column 'White_attacked'")
df['White_attacked'] = df['Board'].progress_apply(lambda board: get_attacked_pieces(board, chess.WHITE))
tqdm.pandas(desc="Adding column 'Black_attacked'")
df['Black_attacked'] = df['Board'].progress_apply(lambda board: get_attacked_pieces(board, chess.BLACK))

df["Attacked_difference"] = df['Black_attacked'] - df['White_attacked']
print("Added column 'Attacked_difference' ✓")

Adding column 'Black_attacked': 100%|██████████| 12959/12959 [00:00<00:00, 13183.11it/s]

Added column 'Attacked_difference' ✓


- **Check:** Verifies if a player's king is under attack

In [113]:
def get_check(board):
    return int(board.is_check()) * 1 if board.turn else -1

tqdm.pandas(desc="Adding column 'Check'")
df['Check'] = df['Board'].progress_apply(get_check)

Adding column 'Check': 100%|██████████| 12959/12959 [00:00<00:00, 315844.44it/s]


<small>*"In chess, doubled pawns are two pawns of the same color residing on the same file. Pawns can become doubled only when one pawn captures onto a file on which another friendly pawn resides. [...] In most cases, doubled pawns are considered a weakness due to their inability to defend each other." [²](#references)*</small>


* **Doubled pawns**: It is the count of how many extra pawns are stacked in each file for a player

In [114]:
def count_doubled_pawns(board, color):
    doubled_pawns = 0
    for file in chess.FILE_NAMES:
        column_squares = [chess.square(chess.FILE_NAMES.index(file), rank) for rank in range(8)]
        pawns_in_file = sum(1 for square in column_squares if board.piece_at(square) == chess.Piece(chess.PAWN, color))
        if pawns_in_file > 1:
            doubled_pawns += pawns_in_file - 1
    return doubled_pawns

tqdm.pandas(desc="Adding column 'White_doubled_pawns'")
df['White_doubled_pawns'] = df['Board'].progress_apply(lambda board: count_doubled_pawns(board, chess.WHITE))
tqdm.pandas(desc="Adding column 'Black_doubled_pawns'")
df['Black_doubled_pawns'] = df['Board'].progress_apply(lambda board: count_doubled_pawns(board, chess.BLACK))

df["Doubled_pawns_difference"] = df['White_doubled_pawns'] - df['Black_doubled_pawns']
print("Added column 'Doubled_pawns_difference' ✓")


Adding column 'Black_doubled_pawns': 100%|██████████| 12959/12959 [00:01<00:00, 7858.70it/s]

Added column 'Doubled_pawns_difference' ✓


<small>*"In chess, a fork is a tactic in which a piece attacks multiple enemy pieces simultaneously. The attacker usually aims to capture one of the forked pieces" [³](#references)*</small>

- **Forks:** Counts the number of forks for each player.

In [115]:
def get_forks(board, color):
    forks = 0
    for square, piece in board.piece_map().items():
        if piece.color == color:
            attacked_squares = board.attacks(square)
            if len([target for target in attacked_squares if board.piece_at(target) and board.piece_at(target).color != color]) > 1:
                forks += 1

    return forks

tqdm.pandas(desc="Adding column 'White_forks'")
df['White_forks'] = df['Board'].progress_apply(lambda board: get_forks(board, chess.WHITE))
tqdm.pandas(desc="Adding column 'Black_forks'")
df['Black_forks'] = df['Board'].progress_apply(lambda board: get_forks(board, chess.BLACK))

df["Forks_difference"] = df['White_forks'] - df['Black_forks']
print("Added column 'Forks_difference' ✓")

Adding column 'Black_forks': 100%|██████████| 12959/12959 [00:01<00:00, 8275.24it/s]

Added column 'Forks_difference' ✓


Let's see the current strongest correlations: 

In [116]:
correlations = df.corr(numeric_only=True)['Evaluation']
sorted_correlations = correlations.reindex(correlations.abs().sort_values(ascending=False).index)
print("Correlations with 'Evaluation' (sorted by absolute value):")
print(sorted_correlations)

Correlations with 'Evaluation' (sorted by absolute value):
Evaluation                    1.000000
Attacks_difference            0.232247
Material_difference           0.195666
Legal_moves_difference        0.178131
Central_attacks_difference    0.121247
White_legal_moves             0.096346
Black_legal_moves            -0.082910
Black_central_attacks        -0.069987
White_attacks                 0.069455
White_central_attacks         0.066475
White_pieces_count            0.052898
Black_attacks                -0.051794
White_material                0.036804
Black_pieces_count           -0.031224
White_forks                   0.030342
En_passant                    0.028921
Turn                          0.028843
Attacked_difference           0.026411
Black_attacked                0.025376
Forks_difference              0.024751
Rule_50                       0.021402
Doubled_pawns_difference      0.021246
Black_doubled_pawns          -0.017758
Black_material               -0.017215
White

Using the chess library, we can extract much more information, but we must keep it simple.  

So far, we have focused on attributes that indicate whether a position might be better, but we have ignored the fact that the goal is to infer a specific value: the evaluation provided by Stockfish 11 at 22 levels of depth (considering 22 turns ahead).  

Therefore, we need to identify the features that Stockfish use to generate their evaluations.

Stockfish's evaluation function is actually quite complex and uses lots of values to determine its result. We'll focus on the main ones and simplify where needed:

- Middle game evaluation

    - **Piece Value (eg)**: Material is weighted differently during the middlegame.

    - **Psqt (eg)**: Piece-square table bonuses, where certain positions provide an advantage for specific pieces in the middlegame.

    - Pawns:
        - Doubled: *(Simplified: Already considered in 'Double_pawns')*

    - Pieces:
        - **Minor behind pawn**: Knight or bishop when behind a pawn
        - **Bishop pawns**: Number of pawns on the same color square as the bishop *(Simplified)*
        - **Bishop xray pawns**: Advantatge for every opponent pawn in the diagonal of a bishop
        - **Rook on queen file**: Simple bonus for a rook that is on the same file as any queen
        - **Rook on file**: Rook when on an open file *(Simplified)*
        - **Long diagonal bishop**: Bonus for bishop on a long diagonal which can "see" both center squares.

    - Mobility: *(Simplified: Already considered in 'Legal_moves')*

    - Threats
        - **Hanging**: Weak enemies not defended by opponent *(Simplified)*

    - **Passed**: Bonuses for passed pawns *(Simplified)*

    - **King**: Bonuses for player's pieces around the king *(Simplified)* 

- End game evaluation

    - **Piece Value (EG)**: Material is weighted differently during the endgame.

    - **Psqt (EG)**: Piece-square table bonuses, where certain positions provide an advantage for specific pieces in the endgame.

    It also takes all the above into account, with different weights *(Will be ignored for simplicity)*

- **Phase**: Eval based on the amount of non-pawn material on the board *(Simplified)*

- Rule50: *(Already considered in 'Rule_50')*

- Scale factor: The scale factors are used to scale the endgame evaluation score down *(It will also be ignored)*

- Tempo: *(Already considered in 'Turn')*

<small>*All the bold titles represent new attributes that will be added.*</small>

In [117]:
# This board representation will be used to do calculations quicker

def fen_to_numpy(fen):
    rows = fen.split(' ')[0].split('/')
    np_board = np.full((8, 8), ' ', dtype=str)
    for i, row in enumerate(rows):
        col = 0
        for char in row:
            if char.isdigit():
                col += int(char)
            else:
                np_board[i, col] = char
                col += 1
                
    return np_board

tqdm.pandas(desc="Adding column 'Numpy_board'")
df['Numpy_board'] = df['FEN'].progress_apply(fen_to_numpy)

Adding column 'Numpy_board': 100%|██████████| 12959/12959 [00:00<00:00, 41529.38it/s]


In [118]:
piece_values_mg = {
    'P': 124, 'N': 781, 'B': 825, 'R': 1276, 'Q': 2538,
    'p': -124, 'n': -781, 'b': -825, 'r': -1276, 'q': -2538
}

def get_white_material_mg(fen):
    return sum(piece_values_mg[piece] for piece in re.findall(r'[QRBNP]', fen))

def get_black_material_mg(fen):
    return -sum(piece_values_mg[piece] for piece in re.findall(r'[qrbnp]', fen))

tqdm.pandas(desc="Adding column 'White_material_mg'")
df['White_material_mg'] = df['FEN'].progress_apply(get_white_material_mg)

tqdm.pandas(desc="Adding column 'Black_material_mg'")
df['Black_material_mg'] = df['FEN'].progress_apply(get_black_material_mg)

df["Material_mg_difference"] = df['White_material_mg'] - df['Black_material_mg']
print("Added column 'Material_mg_difference' ✓")

Adding column 'Black_material_mg': 100%|██████████| 12959/12959 [00:00<00:00, 109824.67it/s]

Added column 'Material_mg_difference' ✓


In [119]:
piece_values_eg = {
    'P': 206, 'N': 854, 'B': 915, 'R': 1380, 'Q': 2682,
    'p': -206, 'n': -854, 'b': -915, 'r': -1380, 'q': -2682
}

def get_white_material_eg(fen):
    return sum(piece_values_eg[piece] for piece in re.findall(r'[QRBNP]', fen))

def get_black_material_eg(fen):
    return -sum(piece_values_eg[piece] for piece in re.findall(r'[qrbnp]', fen))

tqdm.pandas(desc="Adding column 'White_material_eg'")
df['White_material_eg'] = df['FEN'].progress_apply(get_white_material_eg)

tqdm.pandas(desc="Adding column 'Black_material_eg'")
df['Black_material_eg'] = df['FEN'].progress_apply(get_black_material_eg)

df["Material_eg_difference"] = df['White_material_eg'] - df['Black_material_eg']
print("Added column 'Material_eg_difference' ✓")

Adding column 'Black_material_eg': 100%|██████████| 12959/12959 [00:00<00:00, 111715.81it/s]

Added column 'Material_eg_difference' ✓


These are the positional weights that Stockfish uses for each piece:

In [120]:
psqt_mg = np.array([[[0, 0, 0, 0, 0, 0, 0, 0],  [3, 3, 10, 19, 16, 19, 7, -5],  [-9, -15, 11, 15, 32, 22, 5, -22],  [-4, -23, 6, 20, 40, 17, 4, -8],  [13, 0, -13, 1, 11, -2, -13, 5],  [5, -12, -7, 22, -8, -5, -15, -8],  [-7, 7, -3, -13, 5, -16, 10, -8],  [0, 0, 0, 0, 0, 0, 0, 0]],
 [[-175, -92, -74, -73, -73, -74, -92, -175],  [-77, -41, -27, -15, -15, -27, -41, -77],  [-61, -17, 6, 12, 12, 6, -17, -61],  [-35, 8, 40, 49, 49, 40, 8, -35],  [-34, 13, 44, 51, 51, 44, 13, -34],  [-9, 22, 58, 53, 53, 58, 22, -9],  [-67, -27, 4, 37, 37, 4, -27, -67],  [-201, -83, -56, -26, -26, -56, -83, -201]],
 [[-53, -5, -8, -23, -23, -8, -5, -53],  [-15, 8, 19, 4, 4, 19, 8, -15],  [-7, 21, -5, 17, 17, -5, 21, -7],  [-5, 11, 25, 39, 39, 25, 11, -5],  [-12, 29, 22, 31, 31, 22, 29, -12],  [-16, 6, 1, 11, 11, 1, 6, -16],  [-17, -14, 5, 0, 0, 5, -14, -17],  [-48, 1, -14, -23, -23, -14, 1, -48]],
 [[-31, -20, -14, -5, -5, -14, -20, -31],  [-21, -13, -8, 6, 6, -8, -13, -21],  [-25, -11, -1, 3, 3, -1, -11, -25],  [-13, -5, -4, -6, -6, -4, -5, -13],  [-27, -15, -4, 3, 3, -4, -15, -27],  [-22, -2, 6, 12, 12, 6, -2, -22],  [-2, 12, 16, 18, 18, 16, 12, -2],  [-17, -19, -1, 9, 9, -1, -19, -17]],
 [[3, -5, -5, 4, 4, -5, -5, 3],  [-3, 5, 8, 12, 12, 8, 5, -3],  [-3, 6, 13, 7, 7, 13, 6, -3],  [4, 5, 9, 8, 8, 9, 5, 4],  [0, 14, 12, 5, 5, 12, 14, 0],  [-4, 10, 6, 8, 8, 6, 10, -4],  [-5, 6, 10, 8, 8, 10, 6, -5],  [-2, -2, 1, -2, -2, 1, -2, -2]],
 [[271, 327, 271, 198, 198, 271, 327, 271],  [278, 303, 234, 179, 179, 234, 303, 278],  [195, 258, 169, 120, 120, 169, 258, 195],  [164, 190, 138, 98, 98, 138, 190, 164],  [154, 179, 105, 70, 70, 105, 179, 154],  [123, 145, 81, 31, 31, 81, 145, 123],  [88, 120, 65, 33, 33, 65, 120, 88],  [59, 89, 45, -1, -1, 45, 89, 59]]]
)

psqt_eg = np.array([[[0, 0, 0, 0, 0, 0, 0, 0],  [-10, -6, 10, 0, 14, 7, -5, -19],  [-10, -10, -10, 4, 4, 3, -6, -4],  [6, -2, -8, -4, -13, -12, -10, -9],  [10, 5, 4, -5, -5, -5, 14, 9],  [28, 20, 21, 28, 30, 7, 6, 13],  [0, -11, 12, 21, 25, 19, 4, 7],  [0, 0, 0, 0, 0, 0, 0, 0]],
 [[-96, -65, -49, -21, -21, -49, -65, -96],  [-67, -54, -18, 8, 8, -18, -54, -67],  [-40, -27, -8, 29, 29, -8, -27, -40],  [-35, -2, 13, 28, 28, 13, -2, -35],  [-45, -16, 9, 39, 39, 9, -16, -45],  [-51, -44, -16, 17, 17, -16, -44, -51],  [-69, -50, -51, 12, 12, -51, -50, -69],  [-100, -88, -56, -17, -17, -56, -88, -100]],
 [[-57, -30, -37, -12, -12, -37, -30, -57],  [-37, -13, -17, 1, 1, -17, -13, -37],  [-16, -1, -2, 10, 10, -2, -1, -16],  [-20, -6, 0, 17, 17, 0, -6, -20],  [-17, -1, -14, 15, 15, -14, -1, -17],  [-30, 6, 4, 6, 6, 4, 6, -30],  [-31, -20, -1, 1, 1, -1, -20, -31],  [-46, -42, -37, -24, -24, -37, -42, -46]],
 [[-9, -13, -10, -9, -9, -10, -13, -9],  [-12, -9, -1, -2, -2, -1, -9, -12],  [6, -8, -2, -6, -6, -2, -8, 6],  [-6, 1, -9, 7, 7, -9, 1, -6],  [-5, 8, 7, -6, -6, 7, 8, -5],  [6, 1, -7, 10, 10, -7, 1, 6],  [4, 5, 20, -5, -5, 20, 5, 4],  [18, 0, 19, 13, 13, 19, 0, 18]],
 [[-69, -57, -47, -26, -26, -47, -57, -69],  [-55, -31, -22, -4, -4, -22, -31, -55],  [-39, -18, -9, 3, 3, -9, -18, -39],  [-23, -3, 13, 24, 24, 13, -3, -23],  [-29, -6, 9, 21, 21, 9, -6, -29],  [-38, -18, -12, 1, 1, -12, -18, -38],  [-50, -27, -24, -8, -8, -24, -27, -50],  [-75, -52, -43, -36, -36, -43, -52, -75]],
 [[1, 45, 85, 76, 76, 85, 45, 1],  [53, 100, 133, 135, 135, 133, 100, 53],  [88, 130, 169, 175, 175, 169, 130, 88],  [103, 156, 172, 172, 172, 172, 156, 103],  [96, 166, 199, 199, 199, 199, 166, 96],  [92, 172, 184, 191, 191, 184, 172, 92],  [47, 121, 116, 131, 131, 116, 121, 47],  [11, 59, 73, 78, 78, 73, 59, 11]]]
)

In [121]:
def get_psqt(weights, np_board, color):
    pieces = 'PNBRQK' if color == chess.WHITE else 'pnbrqk'
    result = 0
    for index, piece in enumerate(pieces):
        mask = (np_board == piece).astype(int)
        masked = weights[index] * mask
        result += masked.sum().sum()
    return result

tqdm.pandas(desc="Adding column 'White_psqt_mg'")
df['White_psqt_mg'] = df['Numpy_board'].progress_apply(lambda np_board: get_psqt(psqt_mg, np_board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_psqt_mg'")
df['Black_psqt_mg'] = df['Numpy_board'].progress_apply(lambda np_board: get_psqt(psqt_mg, np_board, chess.BLACK))

df["Psqt_mg_difference"] = df['White_psqt_mg'] - df['Black_psqt_mg']
print("Added column 'Psqt_mg_difference' ✓")

Adding column 'White_psqt_mg':   0%|          | 0/12959 [00:00<?, ?it/s]

Adding column 'Black_psqt_mg': 100%|██████████| 12959/12959 [00:01<00:00, 12065.97it/s]

Added column 'Psqt_mg_difference' ✓


In [122]:
tqdm.pandas(desc="Adding column 'White_psqt_eg'")
df['White_psqt_eg'] = df['Numpy_board'].progress_apply(lambda np_board: get_psqt(psqt_eg,np_board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_psqt_eg'")
df['Black_psqt_eg'] = df['Numpy_board'].progress_apply(lambda np_board: get_psqt(psqt_eg,np_board, chess.BLACK))

df["Psqt_eg_difference"] = df['White_psqt_eg'] - df['Black_psqt_eg']
print("Added column 'Psqt_eg_difference' ✓")

Adding column 'Black_psqt_eg': 100%|██████████| 12959/12959 [00:01<00:00, 12341.37it/s]

Added column 'Psqt_eg_difference' ✓


In [123]:
def get_minor_behind_pawn(np_board, color):
    result = 0
    pieces = 'NB' if color == chess.WHITE else 'nb'
    board = np_board if color == chess.WHITE else np_board[::-1,:]
    for piece in pieces:
        idxs = np.argwhere(board == piece)
        idxs = idxs + np.array([-1, 0])
        idxs = idxs[idxs[:, 0] >= 0]
        in_front = board[idxs[:, 0], idxs[:, 1]] == 'P' if color == chess.WHITE else board[idxs[:, 0], idxs[:, 1]] == 'p'
        result += in_front.sum()
    return result

tqdm.pandas(desc="Adding column 'White_minor_behind_pawn'")
df['White_minor_behind_pawn'] = df['Numpy_board'].progress_apply(lambda np_board: get_minor_behind_pawn(np_board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_minor_behind_pawn'")
df['Black_minor_behind_pawn'] = df['Numpy_board'].progress_apply(lambda np_board: get_minor_behind_pawn(np_board, chess.BLACK))

df["Minor_behind_pawn_difference"] = df['White_minor_behind_pawn'] - df['Black_minor_behind_pawn']
print("Added column 'Psqt_mg_difference' ✓")

Adding column 'Black_minor_behind_pawn': 100%|██████████| 12959/12959 [00:01<00:00, 11278.62it/s]

Added column 'Psqt_mg_difference' ✓


In [124]:
def get_bishop_pawns(np_board, color):
    result = 0
    pieces = 'B' if color == chess.WHITE else 'b'
    pawns = 'P' if color == chess.WHITE else 'p'
    board = np_board if color == chess.WHITE else np_board[::-1,:]
    bishop_positions = np.argwhere(board == pieces)
    bishop_colors = (bishop_positions[:, 0] + bishop_positions[:, 1]) % 2
    
    for bishop_color in bishop_colors:
        if bishop_color == 0:
            pawns_on_color = (board[::2, ::2] == pawns).sum()
        else:
            pawns_on_color = (board[1::2, 1::2] == pawns).sum()
        
        result += pawns_on_color
    return result

tqdm.pandas(desc="Adding column 'White_bishop_pawns'")
df['White_bishop_pawns'] = df['Numpy_board'].progress_apply(lambda np_board: get_bishop_pawns(np_board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_bishop_pawns'")
df['Black_bishop_pawns'] = df['Numpy_board'].progress_apply(lambda np_board: get_bishop_pawns(np_board, chess.BLACK))

df["Bishop_pawns_difference"] = df['White_bishop_pawns'] - df['Black_bishop_pawns']
print("Added column 'Bishop_pawns_difference' ✓")


Adding column 'Black_bishop_pawns': 100%|██████████| 12959/12959 [00:00<00:00, 20833.81it/s]

Added column 'Bishop_pawns_difference' ✓


In [125]:
def get_bishop_xray_pawns(np_board, color):
    result = 0
    pieces = 'B' if color == chess.WHITE else 'b'
    pawns = 'P' if color == chess.WHITE else 'p'
    board = np_board if color == chess.WHITE else np_board[::-1,:]
    bishop_positions = np.argwhere(board == pieces)
    for bishop_row, bishop_col in bishop_positions:
        for direction in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
            row, col = bishop_row, bishop_col
            while 0 <= row < board.shape[0] and 0 <= col < board.shape[1]:
                row += direction[0]
                col += direction[1]
                if 0 <= row < board.shape[0] and 0 <= col < board.shape[1]:
                    if board[row, col] == pawns:
                        result += 1
                    if board[row, col] != '.':
                        break

    return result

tqdm.pandas(desc="Adding column 'White_bishop_xray_pawns'")
df['White_bishop_xray_pawns'] = df['Numpy_board'].progress_apply(lambda np_board: get_bishop_xray_pawns(np_board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_bishop_xray_pawns'")
df['Black_bishop_xray_pawns'] = df['Numpy_board'].progress_apply(lambda np_board: get_bishop_xray_pawns(np_board, chess.BLACK))

df["Bishop_xray_pawns_difference"] = df['White_bishop_xray_pawns'] - df['Black_bishop_xray_pawns']
print("Added column 'Bishop_xray_pawns_difference' ✓")

Adding column 'White_bishop_xray_pawns':   0%|          | 0/12959 [00:00<?, ?it/s]

Adding column 'Black_bishop_xray_pawns': 100%|██████████| 12959/12959 [00:00<00:00, 26830.29it/s]

Added column 'Bishop_xray_pawns_difference' ✓


In [126]:
def get_rook_on_queen_file(np_board, color):
    result = 0
    rook_piece = 'R' if color == chess.WHITE else 'r'
    queen_piece = 'Q' if color == chess.WHITE else 'q'
    board = np_board if color == chess.WHITE else np_board[::-1,:]
    rook_positions = np.argwhere(board == rook_piece)
    queen_positions = np.argwhere(board == queen_piece)

    if rook_positions.size > 0 and queen_positions.size > 0:
        rook_cols = rook_positions[:, 1]
        queen_cols = queen_positions[:, 1]
        same_file = np.isin(rook_cols, queen_cols)
        result = same_file.sum()

    return result

tqdm.pandas(desc="Adding column 'White_rook_on_queen_file'")
df['White_rook_on_queen_file'] = df['Numpy_board'].progress_apply(lambda np_board: get_rook_on_queen_file(np_board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_rook_on_queen_file'")
df['Black_rook_on_queen_file'] = df['Numpy_board'].progress_apply(lambda np_board: get_rook_on_queen_file(np_board, chess.BLACK))

df["Rook_on_queen_file_difference"] = df['White_rook_on_queen_file'] - df['Black_rook_on_queen_file']
print("Added column 'Rook_on_queen_file_difference' ✓")

Adding column 'Black_rook_on_queen_file': 100%|██████████| 12959/12959 [00:01<00:00, 12225.39it/s]

Added column 'Rook_on_queen_file_difference' ✓


In [127]:
def get_rook_on_open_file(np_board, color):
    result = 0
    rook_piece = 'R' if color == chess.WHITE else 'r'
    pawns = ['P', 'p']
    board = np_board if color == chess.WHITE else np_board[::-1,:]
    pawn_positions = np.isin(board, pawns)
    files_with_pawns = np.any(pawn_positions, axis=0)
    rook_positions = np.argwhere(board == rook_piece)
    
    if rook_positions.size > 0:
        rook_cols = rook_positions[:, 1]
        open_files = ~files_with_pawns
        rooks_on_open_files = open_files[rook_cols]
        result = rooks_on_open_files.sum()

    return result

tqdm.pandas(desc="Adding column 'White_rook_on_open_file'")
df['White_rook_on_open_file'] = df['Numpy_board'].progress_apply(lambda np_board: get_rook_on_open_file(np_board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_rook_on_open_file'")
df['Black_rook_on_open_file'] = df['Numpy_board'].progress_apply(lambda np_board: get_rook_on_open_file(np_board, chess.BLACK))

df["Rook_on_open_file_difference"] = df['White_rook_on_open_file'] - df['Black_rook_on_open_file']
print("Added column 'Rook_on_open_file_difference' ✓")

Adding column 'Black_rook_on_open_file': 100%|██████████| 12959/12959 [00:01<00:00, 12168.09it/s]

Added column 'Rook_on_open_file_difference' ✓


In [128]:
long_diag = np.array([[1,0,0,0,0,0,0,1],[0,1,0,0,0,0,1,0],[0,0,1,0,0,1,0,0],[0,0,0,1,1,0,0,0],[0,0,0,1,1,0,0,0],[0,0,1,0,0,1,0,0],[0,1,0,0,0,0,1,0],[1,0,0,0,0,0,0,1]]).astype(bool)

In [129]:
def get_long_diagonal_bishop(np_board, color):
    bishop = 'B' if color == chess.WHITE else 'b'
    mask = np_board == bishop
    masked = long_diag * mask
    return masked.sum()

tqdm.pandas(desc="Adding column 'White_long_diagonal_bishop'")
df['White_long_diagonal_bishop'] = df['Numpy_board'].progress_apply(lambda np_board: get_long_diagonal_bishop(np_board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_long_diagonal_bishop'")
df['Black_long_diagonal_bishop'] = df['Numpy_board'].progress_apply(lambda np_board: get_long_diagonal_bishop(np_board, chess.BLACK))

df["Long_diagonal_bishop_difference"] = df['White_long_diagonal_bishop'] - df['Black_long_diagonal_bishop']
print("Added column 'Long_diagonal_bishop_difference' ✓")

Adding column 'Black_long_diagonal_bishop': 100%|██████████| 12959/12959 [00:00<00:00, 46448.18it/s]

Added column 'Long_diagonal_bishop_difference' ✓


In [130]:
def get_hanging_pieces(board, color):
    hanging_count = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece and piece.color == color:
            if not board.is_attacked_by(color, square):
                hanging_count += 1

    return hanging_count

tqdm.pandas(desc="Adding column 'White_hanging_pieces'")
df['White_hanging_pieces'] = df['Board'].progress_apply(lambda board: get_hanging_pieces(board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_hanging_pieces'")
df['Black_hanging_pieces'] = df['Board'].progress_apply(lambda board: get_hanging_pieces(board, chess.BLACK))

df["Hanging_pieces_difference"] = df['Black_hanging_pieces'] - df['White_hanging_pieces']
print("Added column 'Hanging_pieces_difference' ✓")

Adding column 'Black_hanging_pieces': 100%|██████████| 12959/12959 [00:00<00:00, 14810.52it/s]

Added column 'Hanging_pieces_difference' ✓


In [131]:
def get_passed_pawns(np_board, color):
    result = 0
    pawn_piece = 'P' if color == chess.WHITE else 'p'
    opponent_pawns = 'p' if color == chess.WHITE else 'P'
    board = np_board if color == chess.WHITE else np_board[::-1, :]
    pawn_positions = np.argwhere(board == pawn_piece)

    for pawn_row, pawn_col in pawn_positions:
        file_range = range(max(0, pawn_col - 1), min(board.shape[1], pawn_col + 2))
        opposing_pawns_in_path = any(
            opponent_pawns in board[row, file_range].tolist()
            for row in range(pawn_row + 1, board.shape[0])
        )
        if not opposing_pawns_in_path:
            result += 1

    return result

tqdm.pandas(desc="Adding column 'White_passed_pawns'")
df['White_passed_pawns'] = df['Numpy_board'].progress_apply(lambda np_board: get_passed_pawns(np_board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_passed_pawns'")
df['Black_passed_pawns'] = df['Numpy_board'].progress_apply(lambda np_board: get_passed_pawns(np_board, chess.BLACK))

df["Passed_pawn_difference"] = df['White_passed_pawns'] - df['Black_passed_pawns']
print("Added column 'Passed_pawn_difference' ✓")

Adding column 'Black_passed_pawns': 100%|██████████| 12959/12959 [00:01<00:00, 7656.55it/s]

Added column 'Passed_pawn_difference' ✓


In [132]:
def get_king_support(np_board, color):
    king_piece = 'K' if color == chess.WHITE else 'k'
    player_pieces = "PNBRQ" if color == chess.WHITE else "pnbrq"

    king_position = np.argwhere(np_board == king_piece)
    if king_position.size == 0:
        return 0

    king_row, king_col = king_position[0]

    row_start = max(0, king_row - 1)
    row_end = min(np_board.shape[0], king_row + 2)
    col_start = max(0, king_col - 1)
    col_end = min(np_board.shape[1], king_col + 2)
    surrounding_area = np_board[row_start:row_end, col_start:col_end]

    mask = np.isin(surrounding_area, list(player_pieces))
    return np.sum(mask)

tqdm.pandas(desc="Adding column 'White_king_support'")
df['White_king_support'] = df['Numpy_board'].progress_apply(lambda np_board: get_king_support(np_board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_king_support'")
df['Black_king_support'] = df['Numpy_board'].progress_apply(lambda np_board: get_king_support(np_board, chess.BLACK))

df["King_support_difference"] = df['White_king_support'] - df['Black_king_support']
print("Added column 'King_support_difference' ✓")

Adding column 'Black_king_support': 100%|██████████| 12959/12959 [00:01<00:00, 12724.79it/s]

Added column 'King_support_difference' ✓


In [133]:
piece_values_mg = {
    'P': 124, 'N': 781, 'B': 825, 'R': 1276, 'Q': 2538,
    'p': -124, 'n': -781, 'b': -825, 'r': -1276, 'q': -2538
}

def get_white_non_pawn_material(fen):
    return sum(piece_values_mg[piece] for piece in re.findall(r'[QRBN]', fen))

def get_black_non_pawn_material(fen):
    return -sum(piece_values_mg[piece] for piece in re.findall(r'[qrbn]', fen))

tqdm.pandas(desc="Adding column 'White_non_pawn_material'")
df['White_non_pawn_material'] = df['FEN'].progress_apply(get_white_non_pawn_material)

tqdm.pandas(desc="Adding column 'Black_non_pawn_material'")
df['Black_non_pawn_material'] = df['FEN'].progress_apply(get_black_non_pawn_material)
df["Non_pawn_material_difference"] = df['White_non_pawn_material'] - df['Black_non_pawn_material']
print("Added column 'Non_pawn_material_difference' ✓")

Adding column 'Black_non_pawn_material': 100%|██████████| 12959/12959 [00:00<00:00, 129593.83it/s]

Added column 'Non_pawn_material_difference' ✓


In [134]:
correlations = df.corr(numeric_only=True)['Evaluation']
sorted_correlations = correlations.reindex(correlations.abs().sort_values(ascending=False).index)
print("Correlations with 'Evaluation' (sorted by absolute value):")
print(sorted_correlations[:10])

Correlations with 'Evaluation' (sorted by absolute value):
Evaluation                      1.000000
Attacks_difference              0.232247
Passed_pawn_difference          0.218993
Material_difference             0.195666
Legal_moves_difference          0.178131
Material_eg_difference          0.175294
Material_mg_difference          0.157285
Non_pawn_material_difference    0.124782
Central_attacks_difference      0.121247
White_legal_moves               0.096346
Name: Evaluation, dtype: float64


In [136]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
clf = RandomForestRegressor()

X, y = df.drop(columns=[target, 'FEN', 'Board', 'Numpy_board']), df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

r2_score(y_test, y_pred)

0.20861564309068414

## References

1. [Chess Piece Relative Value - Wikipedia](https://en.wikipedia.org/wiki/Chess_piece_relative_value)
2. [Doubled pawns - Wikipedia](https://en.wikipedia.org/wiki/Doubled_pawn)
3. [Fork (chess) - Wikipedia](https://en.wikipedia.org/wiki/Fork_(chess))